This is the main notebook that is used in Airflow. We need to update the name of the file in 0af8d0adfd13_dag.py, which then needs to be uploaded into the right S3 bucket.

This is identical with the notebook called 'test' but we need to change the settings


In [0]:
# import pyspark functions
from pyspark.sql.functions import *
# import URL processing
import urllib

In [0]:
dbutils.fs.unmount("/mnt/user-0af8d0adfd13-bucket") # this will unmount if it is mounted to prevent errors

/mnt/user-0af8d0adfd13-bucket has been unmounted.
Out[55]: True

In [0]:
dbutils.fs.ls("/FileStore/tables")

Out[56]: [FileInfo(path='dbfs:/FileStore/tables/authentication_credentials.csv', name='authentication_credentials.csv', size=202, modificationTime=1687110999000),
 FileInfo(path='dbfs:/FileStore/tables/geo_dirty.csv/', name='geo_dirty.csv/', size=0, modificationTime=1701439735463),
 FileInfo(path='dbfs:/FileStore/tables/pin_dirty.csv/', name='pin_dirty.csv/', size=0, modificationTime=1701439735463),
 FileInfo(path='dbfs:/FileStore/tables/user_dirty.csv/', name='user_dirty.csv/', size=0, modificationTime=1701439735463)]

In [0]:
# Specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

In [0]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [0]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-0af8d0adfd13-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/user-0af8d0adfd13-bucket"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive

def sub_unmount(str_path):
    if any(mount.mountPoint == str_path for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(str_path)

sub_unmount(MOUNT_NAME)

dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

Out[59]: True

Show the files in the topics directory

In [0]:
display(dbutils.fs.ls("/mnt/user-0af8d0adfd13-bucket/topics"))

path,name,size,modificationTime
dbfs:/mnt/user-0af8d0adfd13-bucket/topics/0af8d0adfd13.geo/,0af8d0adfd13.geo/,0,1701439761930
dbfs:/mnt/user-0af8d0adfd13-bucket/topics/0af8d0adfd13.pin/,0af8d0adfd13.pin/,0,1701439761930
dbfs:/mnt/user-0af8d0adfd13-bucket/topics/0af8d0adfd13.user/,0af8d0adfd13.user/,0,1701439761930


In [0]:
from datetime import datetime 
#dbutils.fs.put("/FileStore/my-stuff/my-file.txt", "This is the actual text that will be saved to disk. Like a 'Hello world!' example")

dbutils.fs.put(f"/mnt/user-0af8d0adfd13-bucket/alternative/test{datetime.now()}.txt", f"Worked at {datetime.now()}")

Wrote 36 bytes.
Out[61]: True

Tests with PySpark

Use df.count() to return the total number of rows in the PySpark DataFrame. This function triggers all transformations on the DataFrame to execute.

Use df.distinct().count() to find the number of unique rows in the PySpark DataFrame.

Use len(df.columns) to get the number of columns in the DataFrame.

You can also get the column count using len(df.dtypes) by retrieving all column names and data types as a list of tuples and applying len() on the list.

To count null values in columns, you can use functions like count(when(isnan(column) | col(column).isNull(), column)) for each column to find the number of null, None, or NaN values.

For counting values in a column, use pyspark.sql.functions.count(column) to count non-null values in a specific column. It ignores null/none values.

pyspark.sql.functions.col

Returns a Column based on the given column name.

##Make dataframes

In [0]:
# lets make this a function

def make_dataframe(table_name):
    ''' Makes a dataframe for each of the data buckets in the S3 bucket for pin, geo and user
    
    Argument:   table_name (string)

    Returns:    dataframe
    '''
    # Asterisk(*) indicates reading all the content of the specified file that have .json extension
    file_location = f"/mnt/user-0af8d0adfd13-bucket/topics/0af8d0adfd13.{table_name}/partition=0/*.json" 
    # you need to remember to put in the partition=0 bit!
    file_type = "json"
    # Ask Spark to infer the schema
    infer_schema = "true"
    # Read in JSONs from mounted S3 bucket
    df = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(file_location)
    # Display Spark dataframe to check its content
    # display(df)
    return df



In [0]:
df_pin = make_dataframe("pin")  #for the Pinterest post data
df_geo = make_dataframe("geo")  #for the geolocation data
df_user = make_dataframe("user") #for the user data.

In [0]:
# https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.DataFrame.replace.html
# https://stackoverflow.com/questions/37038014/pyspark-replace-strings-in-spark-dataframe-column#:~:text=The%20function%20withColumn%20is%20called,substrings%20that%20match%20the%20pattern.


def replace_with(df, column, regular_expression, replaced):
    df.groupBy(column).count().orderBy('count', ascending=False).show()
    # df_pin_temp = df_pin.withColumn('description', regexp_replace('description', 'No description available[\W\w]*', None))
    df_temp = df_pin.withColumn(column, regexp_replace(column, regular_expression, None))
    df_temp.groupBy(column).count().orderBy('count', ascending=False).show()
    return df_temp


df_temp = replace_with(df_pin, 'category', 'beaut', None)

+--------------+-----+
 category|count|
+--------------+-----+
 christmas| 186|
 art| 180|
 quotes| 159|
 travel| 154|
 finance| 148|
 mens-fashion| 146|
diy-and-crafts| 146|
 education| 144|
 tattoos| 135|
event-planning| 128|
 home-decor| 122|
 vehicles| 120|
 beauty| 119|
+--------------+-----+

+--------+-----+
category|count|
+--------+-----+
 null| 1887|
+--------+-----+

## Task 1: Clean the Dataframe that contains information about Pinterest posts

To clean the df_pin DataFrame you should perform the following transformations:

- Replace empty entries and entries with no relevant data in each column with Nones ✅ --> but still some tinkering to do
- Perform the necessary transformations on the follower_count to ensure every entry is a number. Make sure the data type of this column is an int. ✅
- Ensure that each column containing numeric data has a numeric data type ✅
- Clean the data in the save_location column to include only the save location path
- Rename the index column to ind. ✅
- Reorder the DataFrame columns to have the following column order:✅
  - ind
  - unique_id
  - title
  - description
  - follower_count
  - poster_name
  - tag_list
  - is_image_or_video
  - image_src
  - save_location
  - category



From researching the data set, the following garbage values were found:

description:  "No description available" x 2 (need to investigate more)

follower_count: "User Info Error"

poster_name: "User Info Error"

image_src: "Image src error."

save_location: We have "Local save in " and then the pathname. Let's see if that all there is, because those are just the top values

tag_list: "N,o, ,T,a,g,s, ,A"...

title: "No Title Data" etc







In [0]:
print('testing')

def clean_column(df, column_name, string, debug=True): # default to None for replacement
    ''' cleans a column of a dataframe, using the LIKE string format. Removes the entire entry and replaces with None
    Arguments:
        df: DataFrame
        column_name: string
        string: string formatted as a LIKE condition
        debug: shows state of the table before and after
    
    '''

    return df.withColumn(column_name,when(df_pin[column_name].like(string), None).otherwise(df_pin[column_name])) 
    


testing

In [0]:

df_pin = clean_column(df_pin, "description", "No description available%")
df_pin = clean_column(df_pin, "follower_count", "User Info Error%")
df_pin = clean_column(df_pin, "poster_name", "User Info Error%")

df_pin = clean_column(df_pin, "image_src", "Image src error%") # prob need to use a regex to keep the right ones here
df_pin = clean_column(df_pin, "save_location", "%.jpg") # still need to have a strategy for random text - need a negative somewhere
df_pin = clean_column(df_pin, "tag_list", "N,o, ,T,a,g,s,%")
df_pin = clean_column(df_pin, "title", "No Title Data%")


In [0]:
def clean_column_regex(df, column_name, regex, replacement, debug=True):
     ''' Replaces the regex the supplied element

     Arguments:
          df: Dataframe
          column_name: string
          regex: regular expression
          replacement: string

     Returns: Dataframe
     '''
     
     return df.withColumn(column_name, regexp_replace(column_name, regex, replacement)) 

 

In [0]:

df_pin = clean_column_regex(df_pin, "save_location", "Local save in ", "") 
df_pin = clean_column_regex(df_pin, "follower_count", "[kK]", "000") 
df_pin = clean_column_regex(df_pin, "follower_count", "[mM]", "000000")
df_pin = df_pin.withColumn("follower_count", df_pin.follower_count.cast("int"))
# df_pin = df_pin.withColumn("index", df_pin.follower_count.cast("int")) # convert to int for index, as was string

clean_column(df_pin, "save_location", "%pinimg%")
clean_column(df_pin, "save_location", "image")

df_pin.printSchema()

root
-- category: string (nullable = true)
-- description: string (nullable = true)
-- downloaded: long (nullable = true)
-- follower_count: integer (nullable = true)
-- image_src: string (nullable = true)
-- index: long (nullable = true)
-- is_image_or_video: string (nullable = true)
-- poster_name: string (nullable = true)
-- save_location: string (nullable = true)
-- tag_list: string (nullable = true)
-- title: string (nullable = true)
-- unique_id: string (nullable = true)

In [0]:
###################START HERE AGAIN
#################################
df_pin = df_pin.withColumnRenamed('index', 'ind') # change name
df_pin = df_pin.select("ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list", "is_image_or_video", "image_src", "save_location", "category") # change order
df_pin.printSchema

Out[69]: <bound method DataFrame.printSchema of DataFrame[ind: bigint, unique_id: string, title: string, description: string, follower_count: int, poster_name: string, tag_list: string, is_image_or_video: string, image_src: string, save_location: string, category: string]>

NEED TO INSERT THE TASK 2 and TASK 3 cleanign code

## Task 2: Clean the Dataframe that contains information about geolocation

To clean the df_geo DataFrame you should perform the following transformations:

- Create a new column coordinates that contains an array based on the latitude and longitude columns
- Drop the latitude and longitude columns from the DataFrame
- Convert the timestamp column from a string to a timestamp data type
- Reorder the DataFrame columns to have the following column order:
  - ind
  - country
  - coordinates
  - timestamp


In [0]:
from pyspark.sql.functions import *

# df_geo = df_geo.withColumn("coordinates", concat(df_geo["latitude"], ', ', df_geo["longitude"])) needs to be an array

df_geo = df_geo.select(df_geo.ind, df_geo.country ,array(df_geo.latitude, df_geo.longitude).alias("coordinates"), df_geo.timestamp)
df_geo = df_geo.withColumn("timestamp", to_timestamp("timestamp"))
df_geo.printSchema()

root
-- ind: long (nullable = true)
-- country: string (nullable = true)
-- coordinates: array (nullable = false)
 |-- element: double (containsNull = true)
-- timestamp: timestamp (nullable = true)

## Task 3: Clean the Dataframe that contains information about Users

To clean the df_user DataFrame you should perform the following transformations:

- Create a new column user_name that concatenates the information found in the first_name and last_name columns
- Drop the first_name and last_name columns from the DataFrame
- Convert the date_joined column from a string to a timestamp data type
- Reorder the DataFrame columns to have the following column order:
  - ind
  - user_name
  - age
  - date_joined

In [0]:
df_user = df_user.withColumn("user_name", concat(df_user["first_name"], lit(" "), df_user["last_name"]))
df_user = df_user.drop('first_name', 'last_name')
df_user = df_user.withColumn("date_joined", to_timestamp("date_joined"))
df_user = df_user.select(df_user.ind, df_user.user_name, df_user.age, df_user.date_joined)
df_user.printSchema


Out[71]: <bound method DataFrame.printSchema of DataFrame[ind: bigint, user_name: string, age: bigint, date_joined: timestamp]>

In [0]:
%sql
-- this doesn't work
-- WITH order_table AS(
--   SELECT date, item, sales, sum(sales)
--     OVER ( PARTITION BY (item) ORDER BY sales DESC ROWS BETWEEN unbounded PRECEDING AND CURRENT ROW) cumsum
--   FROM sales_table
-- ) 

-- SELECT date, item, FIRST(sales)
--   OVER ( PARTITION BY (date) )
-- FROM order_table
-- GROUP BY date, item, sales


-- SELECT DISTINCT country ,--ON (country) country,
--     category,
--     count
-- FROM (
--         SELECT country,
--             category,
--             count(*) AS count
--         FROM pin_table
--             JOIN geo_table ON geo_table.ind = pin_table.index
--         GROUP BY country,
--             category
--         ORDER BY country,
--             count DESC
--     )
-- ORDER BY country,
--     count DESC NULLS LAST;


### QUERIES

In [0]:
# create views to query the data using SQL

df_pin.createOrReplaceTempView("pin_table")
df_geo.createOrReplaceTempView('geo_table')
df_user.createOrReplaceTempView('user_table')
result = spark.sql("SELECT * FROM pin_table")
result.show()

+-----+--------------------+---------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
 ind| unique_id| title| description|follower_count| poster_name| tag_list|is_image_or_video| image_src| save_location| category|
+-----+--------------------+---------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
 4387|ae5e7377-f1bd-4ac...| Βάπτιση: H παραμυ...|Το όνομα που επέλ...| 4|Manosbojana Katsa...|Diy Flowers,Flowe...| image|https://i.pinimg....|/data/event-planning|event-planning|
 6717|bc5ab9ee-505e-44f...|〚 Уютные шведские...|Традиционные швед...| 136000|PUFIK Interiors &...|Scandinavian Cott...| image|https://i.pinimg....| /data/home-decor| home-decor|
 6717|bc5ab9ee-505e-44f...|〚 Уютные шведские...|Традиционные швед...| 136000|PUFIK Interiors &...|Scandinavian Cott...| image|https://i.pinimg....| /data/home-decor| home-decor|
 4858|58101415-9273-431...| THE EVENT COLLECT...|15.1k Likes, 83 C...| 311| Marie Bradford|Diy Birthday Deco...| image|https://i.pinimg....|/data/event-planning|event-planning|
 4608|d234e56f-5b18-4ef...| Virtual Baby Show...|Wow your guests! ...| 1000|Iconica Design | ...|Christmas Party B...| image|https://i.pinimg....|/data/event-planning|event-planning|
 6633|d136f6bc-840d-44f...| The Cottage Journ...|6,636 Likes, 141 ...| 394| Sarah Martin|Diy Kitchen Cabin...| image|https://i.pinimg....| /data/home-decor| home-decor|
 1706|b5c8a1b5-9e90-452...| Standing Figurine...|Features: Materia...| 5000| Wear24-7|Merry Christmas T...| image|https://i.pinimg....| /data/christmas| christmas|
 1706|b5c8a1b5-9e90-452...| Standing Figurine...|Features: Materia...| 5000| Wear24-7|Merry Christmas T...| image|https://i.pinimg....| /data/christmas| christmas|
 1706|b5c8a1b5-9e90-452...| Standing Figurine...|Features: Materia...| 5000| Wear24-7|Merry Christmas T...| image|https://i.pinimg....| /data/christmas| christmas|
 1676|3ed92c2d-9cca-4cc...| PAUPOO™ Pre-lit A...|❤️ ❤️ MERRY CHRIS...| 784| paupoo|Christmas Hanging...| image|https://i.pinimg....| /data/christmas| christmas|
 2482|08604f20-fa17-4b9...| FORNT PORCH CHRIS...|Christmas decorat...| 46000|Life on Summerhil...|Diy Christmas Dec...| video|https://i.pinimg....| /data/christmas| christmas|
 2482|08604f20-fa17-4b9...| FORNT PORCH CHRIS...|Christmas decorat...| 46000|Life on Summerhil...|Diy Christmas Dec...| video|https://i.pinimg....| /data/christmas| christmas|
 2482|08604f20-fa17-4b9...| FORNT PORCH CHRIS...|Christmas decorat...| 46000|Life on Summerhil...|Diy Christmas Dec...| video|https://i.pinimg....| /data/christmas| christmas|
 2057|883542eb-d365-403...| Green/White Pine ...|Sick of sweeping ...| 5000| Wear24-7|Christmas Garden,...| image|https://i.pinimg....| /data/christmas| christmas|
 4712|0eb5569f-0130-455...| My Big Day Market...|Creative Events i...| 10000|My Big Day Market...|Fall Wedding,Drea...| image|https://i.pinimg....|/data/event-planning|event-planning|
 2430|7e16143b-d84a-40a...| 9ft Lighted Winte...|About This Item W...| 5000| Wear24-7|Outside Christmas...| image|https://i.pinimg....| /data/christmas| christmas|
 4348|c25f8906-d9ab-414...| Surprise Birthday...|"adult Surprise B...| 4000000| Etsy|90th Birthday Car...| image|https://i.pinimg....|/data/event-planning|event-planning|
 6521|dd508c7e-4ff3-4a9...| Allan Copley Desi...|Description The C...| 137|Your Home, Reimag...|Glass Top End Tab...| image|https://i.pinimg....| /data/home-decor| home-decor|
 6521|dd508c7e-4ff3-4a9...| Allan Copley Desi...|Description The C...| 137|Your Home, Reimag...|Glass Top End Tab...| image|https://i.pinimg....| /data/home-decor| home-decor|
10538|5d9fa7e2-2118-444...| BC Customs (BCC) ...|By David Crane ; ...| 709| Ricky Lee|Army Vehicles,Arm...| image|https://i.pinimg....| /data/vehicles| vehicles|
+-----+-------------

THIS ONE WORKS!!!!!!

###TASK 4: Find the most popular Pinterest category people post to based on their country.


Your query should return a DataFrame that contains the following columns:

country
category
category_count, a new column containing the desired query output

In [0]:
# First we need to create views which the SQL can access
df_pin.createOrReplaceTempView("pin_table")
df_geo.createOrReplaceTempView('geo_table')
df_user.createOrReplaceTempView('user_table')

In [0]:
%sql
WITH join_tables AS (
  SELECT country,
  category,
  count(*) AS count
  FROM pin_table
      JOIN geo_table ON geo_table.ind = pin_table.ind
  GROUP BY country,
           category

),
get_rank AS (
  SELECT country, 
  category, 
  count, 
  RANK () OVER (
    PARTITION BY country
    ORDER BY count DESC
  ) cat_rank
FROM 
  join_tables
)
SELECT DISTINCT country, 
  category, 
  count
FROM 
  get_rank
WHERE 
  cat_rank=1


  -- ORDER BY country,
  --     count DESC

country,category,count
Afghanistan,education,25
Albania,art,37
Algeria,quotes,50
American Samoa,education,18
Andorra,tattoos,12
Angola,diy-and-crafts,15
Anguilla,diy-and-crafts,16
Antarctica (the territory South of 60 deg S),christmas,12
Antigua and Barbuda,art,18
Argentina,tattoos,23


In [0]:
#YOU NEED TO RUN THIS IMMEDIATELY AFTER THE SQL QUERY TO KEEP IT
task4_df = _sqldf 
task4_df.show()

+--------------------+--------------+-----+
 country| category|count|
+--------------------+--------------+-----+
 Afghanistan| education| 25|
 Albania| art| 37|
 Algeria| quotes| 50|
 American Samoa| education| 18|
 Andorra| tattoos| 12|
 Angola|diy-and-crafts| 15|
 Anguilla|diy-and-crafts| 16|
Antarctica (the t...| christmas| 12|
 Antigua and Barbuda| art| 18|
 Argentina| tattoos| 23|
 Armenia|diy-and-crafts| 19|
 Aruba| christmas| 21|
 Australia| mens-fashion| 19|
 Austria| travel| 20|
 Azerbaijan| finance| 16|
 Bahamas| art| 11|
 Bahrain| finance| 6|
 Bangladesh| christmas| 15|
 Barbados| travel| 17|
 Belarus| travel| 2|
+--------------------+--------------+-----+
only showing top 20 rows


## Task 5 Find how many posts each category had between 2018 and 2022.

Your query should return a DataFrame that contains the following columns:

post_year, a new column that contains only the year from the timestamp column

category

category_count, a new column containing the desired query output



In [0]:
df_pin.show(1)
df_geo.show(1)
df_user.show(1)

+----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
 ind| unique_id| title| description|follower_count| poster_name| tag_list|is_image_or_video| image_src| save_location| category|
+----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
4387|ae5e7377-f1bd-4ac...|Βάπτιση: H παραμυ...|Το όνομα που επέλ...| 4|Manosbojana Katsa...|Diy Flowers,Flowe...| image|https://i.pinimg....|/data/event-planning|event-planning|
+----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
only showing top 1 row

+----+--------------------+--------------------+-------------------+
 ind| country| coordinates| timestamp|
+----+--------------------+--------------------+-------------------+
9455|British Indian Oc...|[-82.9272, -150.346]|2022-03-15 01:46:32|
+----+--------------------+--------------------+-------------------+
only showing top 1 row

+----+--------------------+---+-------------------+
 ind| user_name|age| date_joined|
+----+--------------------+---+-------------------+
6353|Christopher Herna...| 42|2017-02-18 00:31:22|
+----+--------------------+---+-------------------+
only showing top 1 row


Ok, so we need to join df_pin and df_geo
then we need to extract the year from the 

In [0]:
%sql
SELECT 
  EXTRACT(YEAR FROM timestamp) AS year,
  category,
  count(category) AS category_count
FROM 
  pin_table
JOIN 
  geo_table ON geo_table.ind = pin_table.ind
WHERE 
  EXTRACT(YEAR FROM timestamp) BETWEEN 2018 AND 2022
GROUP BY 
  year, category
ORDER BY 
  year, category


year,category,category_count
2018,art,52
2018,beauty,42
2018,christmas,60
2018,diy-and-crafts,64
2018,education,60
2018,event-planning,23
2018,finance,33
2018,home-decor,28
2018,mens-fashion,47
2018,quotes,71


In [0]:
## run after Task 5 SQL to access in Databricks
task5_df = _sqldf

### Task 6: Find the user with the most followers in each country

####Step 1: For each country find the user with the most followers.

Your query should return a DataFrame that contains the following columns:

- country
- poster_name
- follower_count

####Step 2: Based on the above query, find the country with the user with most followers.

Your query should return a DataFrame that contains the following columns:

- country
- follower_count

This DataFrame should have only one entry.


So, we need to join user and geo table, 
and then we partition then table to rank the number of users for each country

In [0]:
df_pin.show(1)

+----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
 ind| unique_id| title| description|follower_count| poster_name| tag_list|is_image_or_video| image_src| save_location| category|
+----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
4387|ae5e7377-f1bd-4ac...|Βάπτιση: H παραμυ...|Το όνομα που επέλ...| 4|Manosbojana Katsa...|Diy Flowers,Flowe...| image|https://i.pinimg....|/data/event-planning|event-planning|
+----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
only showing top 1 row

In [0]:
%sql
--Task 6, Part 1

WITH initial_table AS (
  SELECT 
    country, 
    user_name AS poster_name,
    follower_count,
    RANK() OVER (
      PARTITION BY country
      ORDER BY follower_count DESC
    ) rank_order
  FROM 
    geo_table 
      JOIN user_table ON user_table.ind = geo_table.ind
      JOIN pin_table ON pin_table.ind = user_table.ind
)

SELECT DISTINCT country,
  poster_name,
  follower_count
FROM 
  initial_table
WHERE
  rank_order = 1;




country,poster_name,follower_count
Afghanistan,Amanda Carlson,3000000
Albania,Aaron Anderson,5000000
Algeria,Cynthia Adams,5000000
American Samoa,Abigail Bates,8000000
Andorra,Alison Bell,1000000
Angola,April Brown,8000000
Anguilla,Corey Andrews,92000
Antarctica (the territory South of 60 deg S),Benjamin Campbell,1000000
Antigua and Barbuda,Ann Chung,1000000
Argentina,Diana Barry,2000000


In [0]:
country_user_table_df = _sqldf
country_user_table_df.createOrReplaceTempView('country_user_table')

In [0]:
%sql

--Task 6, Part 2
-- Find the country with the user with the most followers
WITH top_country_user AS (
  SELECT 
    country, 
    follower_count
  FROM 
    country_user_table
  GROUP BY follower_count, country
  ORDER BY follower_count DESC
  LIMIT 1
)
SELECT country,
  follower_count
FROM 
  top_country_user;

  -- this will output only one entry as requested. However, there may be more than one country that has the highest number of users if there is a draw. If we wanted that, we would have used RANK()



country,follower_count
American Samoa,8000000


In [0]:
task6_part2_df = _sqldf

### Task 7: Find the most popular category for different age groups

What is the most popular category people post to based on the following age groups:

  - 18-24
  - 25-35
  - 36-50
  - +50

Your query should return a DataFrame that contains the following columns:

  - age_group, a new column based on the original age column
  - category
  category_count, a new column containing the desired query output

Category is in pin_table and age is in user, so we need to join those tables
Then we need to create a temporary table where we assign someone a category of one of 4 ages
Then we partition the table based on these categories and create a "category count" column that we rank descending,
and we return where rank is 1

In [0]:
%sql
--first subquery is to create the age group 
-- WITH create_age_cat AS
--   (
--     SELECT category,
--     CASE
--       WHEN age >= 18 AND age <=24 THEN '18-24'
--       WHEN age >= 25 AND age <=35 THEN '25-35'
--       WHEN age >= 36 AND age <=50 THEN '36-50'
--       WHEN age > 50 THEN '+50'
--       ELSE NULL
--     END 
--       AS age_group
--   FROM pin_table JOIN user_table ON pin_table.ind = user_table.ind
--   ),
--   count_cat AS 
--   (
--     SELECT age_group, 
--       COUNT(category) AS category_count
--     FROM 
--       create_age_cat
--     GROUP BY 
--       age_group
--   )


-- SELECT age_group, 
--   category_count,
--   RANK() OVER (PARTITION BY age_group ORDER BY category_count)
-- FROM count_cat;  


WITH create_age_cat AS
  (
    SELECT category,
    CASE
      WHEN age >= 18 AND age <=24 THEN '18-24'
      WHEN age >= 25 AND age <=35 THEN '25-35'
      WHEN age >= 36 AND age <=50 THEN '36-50'
      WHEN age > 50 THEN '50+'
      ELSE NULL
    END 
      AS age_group
  FROM pin_table JOIN user_table ON pin_table.ind = user_table.ind
  ),
  count_cat AS (
    SELECT age_group, category,
      COUNT(category) AS category_count
    FROM 
      create_age_cat
    GROUP BY 
      age_group, category
  ),
  ranked_table AS (
    SELECT age_group, category,
    category_count,
    RANK() OVER (PARTITION BY age_group ORDER BY category_count DESC) ranked
  FROM  
    count_cat
  )

  SELECT
   age_group, 
   category,
   category_count
  FROM 
    ranked_table
  WHERE 
    ranked = 1
  ORDER BY age_group;
  
  
  





age_group,category,category_count
18-24,art,165
25-35,travel,94
36-50,quotes,84
50+,beauty,25
50+,vehicles,25


In [0]:
task7_df = _sqldf

### Task 8: Find the median follower count for the different age groups

What is the median follower count for users in the following age groups:

- 18-24
- 25-35
- 36-50
- 50+

Your query should return a DataFrame that contains the following columns:

- age_group, a new column based on the original age column
- median_follower_count, a new column containing the desired query output

We divide the follower counts into age categories using a case statement.
Then we count the rows per partition and divide the total of followers by the total (is the median?) If odd number, it is the middle of the list. If even, it is the middle two numbers divided by 2.

*** We actually used percentile to calculate this. Not the best, but it is approx and i think the data needs more cleaning.



In [0]:
%sql
WITH follower_table AS (
  SELECT follower_count,
    CASE
      WHEN age >= 18 AND age <=24 THEN '18-24'
      WHEN age >= 25 AND age <=35 THEN '25-35'
      WHEN age >= 36 AND age <=50 THEN '36-50'
      WHEN age > 50 THEN '50+'
      ELSE NULL
    END 
      AS age_group
  FROM pin_table JOIN user_table ON pin_table.ind = user_table.ind
)

SELECT DISTINCT age_group, 
  --PERCENTILE_CONT(.5) WITHIN GROUP (ORDER BY follower_count) OVER(PARTITION BY age_group) median_follower_count
  approx_percentile(follower_count, 0.5) OVER(PARTITION BY age_group) median_follower_count
  
FROM
  follower_table;



age_group,median_follower_count
18-24,92000
25-35,27000
36-50,7000
50+,1000


In [0]:
task8_df = _sqldf

In [0]:
%sql
-- to check result the figures above should be the middle rows of this outpit:
SELECT follower_count,
    CASE
      WHEN age >= 18 AND age <=24 THEN '18-24'
      WHEN age >= 25 AND age <=35 THEN '25-35'
      WHEN age >= 36 AND age <=50 THEN '36-50'
      WHEN age > 50 THEN '50+'
      ELSE NULL
    END 
      AS age_group
  FROM pin_table JOIN user_table ON pin_table.ind = user_table.ind
  ORDER BY age_group, follower_count

follower_count,age_group
null,18-24
null,18-24
null,18-24
null,18-24
null,18-24
null,18-24
null,18-24
null,18-24
null,18-24
null,18-24


### Task 9: Find how many users have joined each year

Find how many users have joined between 2015 and 2020.

Your query should return a DataFrame that contains the following columns:

- post_year, a new column that contains only the year from the timestamp column
- number_users_joined, a new column containing the desired query output

In [0]:
%sql
SELECT EXTRACT(YEAR FROM date_joined) AS post_year,
    COUNT(user_name) AS number_users_joined
FROM 
    user_table
GROUP BY 
    EXTRACT(YEAR FROM date_joined)

post_year,number_users_joined
2015,735
2016,846
2017,307


### Task 10: Find the median follower count of users based on their joining year
Find the median follower count of users have joined between 2015 and 2020.

Your query should return a DataFrame that contains the following columns:

- post_year, a new column that contains only the year from the timestamp column
- median_follower_count, a new column containing the desired query output


so, we need to look at all the follower counts based on user as they are divided by year, and then find the median for that

In [0]:
df_pin.show(1)
df_user.show(1)
df_geo.show(1)

+----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
 ind| unique_id| title| description|follower_count| poster_name| tag_list|is_image_or_video| image_src| save_location| category|
+----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
4387|ae5e7377-f1bd-4ac...|Βάπτιση: H παραμυ...|Το όνομα που επέλ...| 4|Manosbojana Katsa...|Diy Flowers,Flowe...| image|https://i.pinimg....|/data/event-planning|event-planning|
+----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
only showing top 1 row

+----+--------------------+---+-------------------+
 ind| user_name|age| date_joined|
+----+--------------------+---+-------------------+
6353|Christopher Herna...| 42|2017-02-18 00:31:22|
+----+--------------------+---+-------------------+
only showing top 1 row

+----+--------------------+--------------------+-------------------+
 ind| country| coordinates| timestamp|
+----+--------------------+--------------------+-------------------+
9455|British Indian Oc...|[-82.9272, -150.346]|2022-03-15 01:46:32|
+----+--------------------+--------------------+-------------------+
only showing top 1 row

In [0]:
%sql
WITH year_table AS (
  SELECT EXTRACT(YEAR FROM date_joined) AS post_year,
    follower_count
  FROM 
    user_table JOIN pin_table ON user_table.ind == pin_table.ind
)

SELECT DISTINCT post_year,
 approx_percentile(follower_count, 0.5) OVER(PARTITION BY post_year) median_follower_count
FROM
  year_table


post_year,median_follower_count
2015,104000
2016,19000
2017,4000


In [0]:
%sql

--Check for the median values NULL seems to be an issue for 2015

 SELECT EXTRACT(YEAR FROM date_joined) AS post_year,
    follower_count
  FROM 
    user_table JOIN pin_table ON user_table.ind == pin_table.ind
  ORDER BY 
    post_year, follower_count

post_year,follower_count
2015,null
2015,null
2015,null
2015,null
2015,null
2015,null
2015,null
2015,null
2015,null
2015,null


### Task 11

Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of.

Your query should return a DataFrame that contains the following columns:

- age_group, a new column based on the original age column
- post_year, a new column that contains only the year from the timestamp column
- median_follower_count, a new column containing the desired query output

In [0]:
%sql

WITH follower_table AS (
  SELECT follower_count,
  EXTRACT(YEAR FROM date_joined) AS post_year,
    CASE
      WHEN age >= 18 AND age <=24 THEN '18-24'
      WHEN age >= 25 AND age <=35 THEN '25-35'
      WHEN age >= 36 AND age <=50 THEN '36-50'
      WHEN age > 50 THEN '50+'
      ELSE NULL
    END 
      AS age_group
  FROM pin_table JOIN user_table ON pin_table.ind = user_table.ind
)

SELECT DISTINCT post_year, age_group, 
  --PERCENTILE_CONT(.5) WITHIN GROUP (ORDER BY follower_count) OVER(PARTITION BY age_group) median_follower_count
  approx_percentile(follower_count, 0.5) OVER(PARTITION BY age_group, post_year) median_follower_count
FROM
  follower_table
ORDER BY
  post_year, age_group;


post_year,age_group,median_follower_count
2015,18-24,221000
2015,25-35,42000
2015,36-50,13000
2015,50+,14000
2016,18-24,40000
2016,25-35,25000
2016,36-50,9000
2016,50+,542
2017,18-24,10000
2017,25-35,6000


In [0]:
%sql
-- Test the results

  SELECT follower_count,
  EXTRACT(YEAR FROM date_joined) AS post_year,
    CASE
      WHEN age >= 18 AND age <=24 THEN '18-24'
      WHEN age >= 25 AND age <=35 THEN '25-35'
      WHEN age >= 36 AND age <=50 THEN '36-50'
      WHEN age > 50 THEN '50+'
      ELSE NULL
    END 
      AS age_group
  FROM pin_table JOIN user_table ON pin_table.ind = user_table.ind
  ORDER BY post_year, age_group, follower_count


follower_count,post_year,age_group
null,2015,18-24
null,2015,18-24
null,2015,18-24
null,2015,18-24
null,2015,18-24
null,2015,18-24
null,2015,18-24
null,2015,18-24
null,2015,18-24
null,2015,18-24


In [0]:
# from pyspark.sql.window import Window

# #result = result.createOrReplaceTempView("result_table")

# # result = spark.sql("SELECT DISTINCT ON (country) country, category, count FROM result_table ORDER BY country, count DESC NULLS LAST;")

# window = Window.partitionBy("country").orderBy("count")

# # res = result.withColumn("country").over(window)
# result.withColumn("rank", rank().over(window)).withColumn("max", max('count').over(window)).sort(desc("count")).show()
# #res.show()
# result.select("country", "category", first(count))


In [0]:
# result.select("country", "category", first(count))